## 0. Imports
Run cell below to import necessary libraries and functions


In [1]:
# Third-party imports
import numpy as np
from aicsimageio import AICSImage

# Local imports
from timeseries_alignment.timeseries_alignment_framework import (
    align_time_series, create_template_hough, ht_rotation_wrapper, 
    ht_prev_rotation_wrapper, orb_rotation_wrapper, orb_prev_rotation_wrapper, 
    orb_translation_wrapper, xcorr_translation_wrapper, align_time_series_multiprocessing
)
from timeseries_alignment.utils import (
    apply_transformations,
    measure_runtime, measure_memory
)

## 1. Dataset Loading
Adjust file path (`img_path`) to point to your dataset


In [2]:
# Dataset selection: adjust path to your data location

# sample experimental dataset of 10 timepoints
img_path = 'data/sample10timepoints.tif'    

# Load image:
img = AICSImage(img_path)
img_data = img.data

## 2. Template Creation

Adjust channel index (`channel`) - ensure it's the phase contrast channel

*Note: This step could be skipped and template creation done directly in Step 3*

In [3]:
# Select channel for template creation (typically phase contrast)
channel = 0

# Create template using HT rotation detection
print("Creating template from first timepoint...")
template, detected_angle = create_template_hough(
    img_data, 
    channel=channel, 
    plot=False, 
    t=0
)

print(f"✓ Template created successfully!")
print(f"✓ Detected rotation angle: {detected_angle:.2f}°")

Creating template from first timepoint...
✓ Template created successfully!
✓ Detected rotation angle: 1.09°


## 3. Run Image Registration

Choose from rotation/translation methods (`rotation_method`/`translation_method`) mentioned below, set (`plot`) to True for debugging and quick visual verification

In [4]:
# Available registration methods:
# Rotation methods:
#   - ht_rotation_wrapper: HT-based rotation
#   - ht_prev_rotation_wrapper: HT-based rotation with temporal initialization
#   - orb_rotation_wrapper: ORB-based rotation
#   - orb_prev_rotation_wrapper: ORB-based rotation with temporal initialization
# Translation methods:
#   - orb_translation_wrapper: ORB-based translation
#   - xcorr_translation_wrapper: Cross-correlation based translation

# Preferred methods: ht_prev_rotation_wrapper and xcorr_translation_wrapper

# Use channel=0 unless already defined
if 'channel' not in locals():
    channel = 0

# Common parameters for registration
align_params = {
    'channel': channel,
    'template': create_template_hough(img_data, channel=channel, plot=False, t=0),
    'rotation_method': ht_prev_rotation_wrapper,
    'translation_method': xcorr_translation_wrapper,
    'plot': False,
    'use_template_prev': True
}

print("Running timeseries alignment...")
detected_shifts, detected_angles = align_time_series(img_data, **align_params)
print(f"✓ Alignment completed! Processed {len(detected_angles)} timepoints")

Running timeseries alignment...
✓ Alignment completed! Processed 10 timepoints


## 4. Save results

Save resulting sample file as tiff in the same folder as the original img (mainly for visual verification, only saves single channel)

In [5]:
apply_transformations(img_data, img_path, detected_shifts, detected_angles, channel=channel)

Transformed image saved to: data/sample10timepoints_aligned.tif


'data/sample10timepoints_aligned.tif'

## 5. Evaluation

Measure runtime and memory usage

In [6]:
# Runtime measurements, 10 runs
runtimes = []
for i in range(10):
    runtime = measure_runtime(align_time_series, img_data, **align_params)
    runtimes.append(runtime)

avg_runtime = np.mean(runtimes)
std_runtime = np.std(runtimes)
print(f"\nAverage runtime: {avg_runtime:.4f} seconds")
print(f"Stddev runtime: {std_runtime:.4f} seconds")

# Memory measurements, 10 runs
memory = []
for i in range(10):
    mem = measure_memory(align_time_series, img_data, **align_params)
    memory.append(mem)

avg_memory = np.mean(memory)
std_memory = np.std(memory)
print(f"\nAverage peak memory: {avg_memory:.4f} MB")
print(f"Stddev peak memory: {std_memory:.4f} MB")

Runtime: 0.2798 seconds
Runtime: 0.2538 seconds
Runtime: 0.2497 seconds
Runtime: 0.2480 seconds
Runtime: 0.2360 seconds
Runtime: 0.2493 seconds
Runtime: 0.2590 seconds
Runtime: 0.2706 seconds
Runtime: 0.2508 seconds
Runtime: 0.2418 seconds

Average runtime: 0.2539 seconds
Stddev runtime: 0.0123 seconds
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB
Peak Memory Usage: 34.00 MB

Average peak memory: 34.0046 MB
Stddev peak memory: 0.0002 MB
